In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from InstructorEmbedding import INSTRUCTOR

In [4]:
# Load the hkunlp/instructor-large model 
model = INSTRUCTOR('hkunlp/instructor-large')


load INSTRUCTOR_Transformer


max_seq_length  512


In [5]:
# Load the csv
dataset_path = csv_file = '../asr/cv-valid-dev.csv'
df = pd.read_csv(dataset_path)

In [17]:
# Define the hot words and their embeddings
hot_words = ["be careful", "destroy", "stranger"]
hot_word_embeddings = {word: model.encode([word]) for word in hot_words}

In [22]:
# Define a function to calculate similarity
def calculate_similarity(transcription):
    sent_embedding = model.encode([transcription])
    similarities = [cosine_similarity(sent_embedding, hot_word_embedding)[0] for hot_word_embedding in hot_word_embeddings.values()]
    return max(similarities)

In [24]:
# Calculate similarity for each phrase in the dataset
similarities = []
for sent in tqdm(df["generated_text"]):
    similarity = calculate_similarity(sent)
    similarities.append(similarity)

100%|██████████| 4076/4076 [01:19<00:00, 51.51it/s]


In [46]:
# Add the similarity column to the dataset
threshold = np.mean(similarities) # can be adjusted
df["similarity"] = [(similarity > threshold).item() for similarity in similarities]

In [47]:
# Save the updated dataset
output_path = "cv-valid-dev-with-similarity.csv"
df.to_csv(output_path, index=False)